In [13]:
%pip show pymilvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: pymilvus
Version: 2.4.4
Summary: Python Sdk for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: /Users/jc/code/AIPI-540-Module-3-Project/.conda/lib/python3.11/site-packages
Requires: environs, grpcio, milvus-lite, pandas, protobuf, setuptools, ujson
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install -q -U "pymilvus[model]"
%pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
from pymilvus import MilvusClient

client = MilvusClient("portugal_ai.db")
COLLECTION_NAME = "portugal_ai"

In [2]:
if client.has_collection(collection_name=COLLECTION_NAME):
    client.drop_collection(collection_name=COLLECTION_NAME)
client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=384,
)

In [3]:
from pymilvus import model

embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='all-MiniLM-L6-v2',
    device='cpu'
)

print("embedding_fn")

/Users/jc/code/AIPI-540-Module-3-Project/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jc/code/AIPI-540-Module-3-Project/.conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


embedding_fn


In [4]:
import os

docs = []
doc_locations = []

# Iterate locations
for location in os.listdir("data"):
    print(location)
    # Iterate files inside location folder
    for file in os.listdir(f"data/{location}"):
        with open(f"data/{location}/{file}", "r") as f:
            content = f.read()
            docs.append(content)
            doc_locations.append(location)  # Save the location of the document
            

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "location": doc_locations[i]}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


santarem
vila real
alentejo
guimaraes
leiria
lisbon
coimbra
covilha
aveiro
algarve
braganca
braga
funchal
azores
porto
viseu
Dim: 384 (384,)
Data has 90 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'location'])
Vector dim: 384


In [5]:
res = client.insert(collection_name=COLLECTION_NAME, data=data)

print(res)

{'insert_count': 90, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89], 'cost': 0}


In [7]:
query_vectors = embedding_fn.encode_queries(["I'm looking for a francesinha, where should I eat?"])

res = client.search(\
    collection_name=COLLECTION_NAME,  # target collection
    data=query_vectors,  # query vectors
    filter="location == 'braga'",  # filter results
    limit=1,  # number of returned entities
    output_fields=["text", "location"],  # specifies fields to be returned
)

for rec in res[0]:
    print(f"Distance: {rec['distance']}")
    print(f"Location: {rec['entity']['location']}")
    print(f"Text: {rec['entity']['text'].splitlines()[0]}")
    print("----")

Distance: 0.4209580421447754
Location: braga
Text: Restaurant Name: Taberna Belga (belgium tavern)
----
